In [ ]:
import os, cv2, sys, toml
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

from opencv_functions import *
from utility_functions import *
from dobot_arm import *

In [ ]:
arm = DobotArm(log_level = 3)
arm.connect()
arm.enable_defaults()

In [ ]:
%matplotlib inline

PATH_TO_CAMERA_CALIBRATION_FILE = "./calibration/20250505-EMEET.npz"
PATH_TO_TEMPLATES = "./templates"

cam = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
cam.set(cv2.CAP_PROP_FOURCC, fourcc)

cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

camera_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
camera_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))

camera_matrix, new_camera_matrix, distortion_coefficients, resolution, focus = get_camera_calibration(PATH_TO_CAMERA_CALIBRATION_FILE)

cam.set(cv2.CAP_PROP_AUTOFOCUS, 0)
cam.set(cv2.CAP_PROP_FOCUS, float(530))

sift, flann = initialize_detector()

# load all template images as numpy arrays
templates = []
template_filenames = [f for f in os.listdir(PATH_TO_TEMPLATES) if f.endswith((".png",
                                                                              ".jpg",
                                                                              ".jpeg",
                                                                              ".bmp",
                                                                              ".tiff"))]

for template_filename in template_filenames:
    path_to_template = os.path.join(PATH_TO_TEMPLATES, template_filename)
    image = cv2.imread(path_to_template)

    if image is not None:
        templates.append(image)

    else:
        print(f"Failed to load template at: {path_to_template}.")

colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]

In [ ]:
ret, frame = cam.read()

best_ious = []
best_transformed_boxes = []
best_thetas = []

if ret:

    # if we have multiple templates, probably zip it with different colors?
    for i, template in enumerate(templates):
        results, metrics = get_detection(sift, flann, template, frame)

        if len(metrics["iou"]) > 0:

            # its slower than [-1], but that only works if we found a good iou early
            idx_best = np.argmax(metrics["iou"])
            best_H = results["H"][idx_best]
            best_transformed_box = results["transformed_box"][idx_best]
            best_iou = metrics["iou"][idx_best]

            best_ious.append(best_iou)
            best_transformed_boxes.append(best_transformed_box)

            if best_transformed_box is not None:
                draw_polygon(frame, best_transformed_box, color = colors[i])
                theta = calculate_matrix_angle(best_H)
                best_thetas.append(theta)

                print(f"Board: {template_filenames[i]}, IOU: {best_iou}, Angle: {theta}")

            else:
                print("Failed to find object.")

        else:
            print("Failed to find object.")

else:
    print("No return.")


plt.imshow(frame)
plt.scatter([1920 / 2], [1080 / 2])
plt.show()

In [ ]:
cam.release()

In [ ]:
PATH_TO_CALIBRATION_FILE = "./calibration/20250505-EMEET.npz"
camera_matrix, new_camera_matrix, distortion_coefficients, resolution, focus = get_camera_calibration(PATH_TO_CALIBRATION_FILE)

best_ious = np.array(best_ious)
best_idx = np.argmax(best_ious)

scale_x = 115 / (1920)
scale_y = 64 / (1080)

object_center = np.mean(best_transformed_boxes[best_idx].reshape(4, 2), axis = 0)

print(camera_matrix[0, 2], camera_matrix[1, 2])
center = np.array([1920 / 2, 1080 / 2])
# center = camera_matrix[:2, 2].ravel()

print(object_center, center)

dx_mm, dy_mm = np.array([scale_x, scale_y]) * (object_center - center)
print(dx_mm, dy_mm)

In [ ]:
# dx_mm, dy_mm = 23.89440155029297, -19.82210922241211
# theta = 108.61350011762714

start_coords = [295.24, -213.2, -43, 25.11]
offset = -35
angle_offset = -best_thetas[best_idx]
arm.movl(start_coords)

if best_iou >= 0.9:
    pass

arm.wait_for_movement_completion()
arm.relmovl([dx_mm + offset, -dy_mm, 0, angle_offset])

arm.wait_for_movement_completion()
print(arm.get_coords())

In [ ]:
arm.open_gripper("limit", 100)
arm.relmovl([0, 0, -47.5, 0])
arm.close_gripper("limit", 25)
arm.relmovl([0, 0, 30, 0])
arm.movl(start_coords)
arm.relmovl([0, 0, -30, 0])
arm.open_gripper("limit", 100)
arm.movl(start_coords)

In [ ]:
start_coords = [295.24, -213.2, -43, 25.11]
arm.movl(start_coords)